# Persistence and Streaming 🌐

## Introduction
This notebook demonstrates the `integration of LangChain's persistence and streaming capabilities` using various tools and APIs. The aim is to build an agent that can fetch data, process it, and manage state efficiently.

## Persistence and Streaming
Persistence and streaming are crucial concepts in the realm of AI and data management. `Persistence` refers to the ability of a system to retain data across sessions and restarts, ensuring that important information is not lost and can be accessed whenever needed. This is typically achieved using databases or file systems. `Streaming`, on the other hand, involves the continuous processing of data as it arrives in real-time, allowing for immediate analysis and response. This is particularly useful in applications such as real-time monitoring, live data feeds, and interactive AI agents. By combining persistence and streaming, systems can offer robust data handling capabilities, ensuring data integrity and providing timely insights.

## Persistence

### Import Required Libraries

In [1]:
# Import necessary modules from the langchain library
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain.chat_models import ChatOpenAI  # Correct import
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
# Import additional required libraries
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

### Load Environment Variables
Load the environment variables from a .env file to access the OpenAI API key.

In [3]:
# Import the required libraries
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()


# Print the Tavily API key (masked)
print(f"TAVILY_API_KEY: {os.getenv('TAVILY_API_KEY')[:5]}*****")

TAVILY_API_KEY: tvly-*****


### Initialize Tavily Search Tool

In [4]:
# Initialize Tavily search tool
tool = TavilySearchResults(max_results=2)

### Define Agent State

In [5]:
# Define the agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

**Note** Ensure the required package is installed for saving checkpoints.

```py
pip install langgraph-checkpoint-sqlite
```

More info on [langgraph-checkpoint-sqlite](https://pypi.org/project/langgraph-checkpoint-sqlite/#files)

In [6]:
# Original code 
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

### Define the Agent Class

In [7]:
# Define the Agent class
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

### Custom Prompt
Define a custom prompt for the LangChain model.

Let's include the original custom prompt within LangChain’s 
````py
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)
````

#### Example Query 1 

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage

# Define the custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.

Only look up information when you are sure of what you want. If you need to look up some information before asking a follow-up question, you are allowed to do that!
"""

# Initialize the Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="TavilySearch",
    func=tool.run,
    description="Search for information using Tavily."
)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the agent with the custom prompt
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  # Gracefully handle parsing errors
)

# Define the messages
messages = [HumanMessage(content="What is the weather in sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Since response is a string, you can print it directly
print(response)

C:\Users\Michela\AppData\Local\Temp\ipykernel_12724\149761321.py:32: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
C:\Users\Michela\AppData\Local\Temp\ipykernel_12724\149761321.py:35: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating fro

The current weather in San Francisco is partly cloudy with a temperature of 55.0°F.
The current weather in San Francisco is partly cloudy with a temperature of 55.0°F.


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage

# Define the custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.

Only look up information when you are sure of what you want. If you need to look up some information before asking a follow-up question, you are allowed to do that!
"""

# Initialize the Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="TavilySearch",
    func=tool.run,
    description="Search for information using Tavily."
)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the agent with the custom prompt
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  # Gracefully handle parsing errors
)

# Define the messages
messages = [HumanMessage(content="What is the weather in sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Since response is a string, you can print it directly
print(response)

The current weather in San Francisco is partly cloudy with a temperature of 55.0°F.
The current weather in San Francisco is partly cloudy with a temperature of 55.0°F.


Taking into consideration the persistence and snippet code 
```py
for event in response.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])
```


- Method 1 

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage
import time

# Define the custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.

Only look up information when you are sure of what you want. If you need to look up some information before asking a follow-up question, you are allowed to do that!
"""

# Initialize the Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="TavilySearch",
    func=tool.run,
    description="Search for information using Tavily."
)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the agent with the custom prompt
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  # Gracefully handle parsing errors
)

# Define the messages
messages = [HumanMessage(content="What is the weather in sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Mock object for streaming events
class MockGraph:
    def stream(self, data, thread):
        for i in range(3):  # Simulate 3 events
            yield {"event": {"values": [{"messages": f"Mock message {i+1}"}]}}
            time.sleep(1)  # Simulate delay between events

class MockStreamingObject:
    def __init__(self):
        self.graph = MockGraph()

# Create an instance of the mock streaming object
streaming_object = MockStreamingObject()

# Example usage with the mock streaming object
for event in streaming_object.graph.stream({"messages": messages}, thread):
    for v in event["event"]["values"]:
        print(v['messages'])

The current weather in San Francisco is partly cloudy with a temperature of 55.0°F, wind speed of 14.3 mph, and humidity of 77%.
Mock message 1
Mock message 2
Mock message 3


- Method 2

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage

# Define the custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.

Only look up information when you are sure of what you want. If you need to look up some information before asking a follow-up question, you are allowed to do that!
"""

# Initialize the Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="TavilySearch",
    func=tool.run,
    description="Search for information using Tavily."
)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the agent with the custom prompt
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  # Gracefully handle parsing errors
)

# Define the messages
messages = [HumanMessage(content="What is the weather in sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Assuming you have a valid object for streaming events
class ValidGraph:
    def stream(self, data, thread):
        # Replace this with actual streaming logic
        for i in range(3):  # Simulate 3 events
            yield {"event": {"values": [{"messages": f"Actual message {i+1}"}]}}
            time.sleep(1)  # Simulate delay between events

class ValidStreamingObject:
    def __init__(self):
        self.graph = ValidGraph()

# Create an instance of the valid streaming object
streaming_object = ValidStreamingObject()

# Example usage with the valid streaming object
for event in streaming_object.graph.stream({"messages": messages}, thread):
    for v in event["event"]["values"]:
        print(v['messages'])

The current temperature in San Francisco is around 49°F.
Actual message 1
Actual message 2
Actual message 3


#### Example Query 2 

In [12]:
# Define the messages
messages = [HumanMessage(content="What about in la?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Assuming you have a valid object for streaming events
class ValidGraph:
    def stream(self, data, thread):
        # Replace this with actual streaming logic
        for i in range(3):  # Simulate 3 events
            yield {"event": {"values": [{"messages": f"Actual message {i+1}"}]}}
            time.sleep(1)  # Simulate delay between events

class ValidStreamingObject:
    def __init__(self):
        self.graph = ValidGraph()

# Create an instance of the valid streaming object
streaming_object = ValidStreamingObject()

# Example usage with the valid streaming object
for event in streaming_object.graph.stream({"messages": messages}, thread):
    for v in event["event"]["values"]:
        print(v['messages'])

Information about "la Los Angeles" can be found on Wikipedia, the official website of the City of Los Angeles, and Discover Los Angeles.
Actual message 1
Actual message 2
Actual message 3


#### Example Query 3 

In [13]:
# Define the messages
#messages = [HumanMessage(content="Which one is warmer?")]
messages = [HumanMessage(content="Which one is warmer la or sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Assuming you have a valid object for streaming events
class ValidGraph:
    def stream(self, data, thread):
        # Replace this with actual streaming logic
        for i in range(3):  # Simulate 3 events
            yield {"event": {"values": [{"messages": f"Actual message {i+1}"}]}}
            time.sleep(1)  # Simulate delay between events

class ValidStreamingObject:
    def __init__(self):
        self.graph = ValidGraph()

# Create an instance of the valid streaming object
streaming_object = ValidStreamingObject()

# Example usage with the valid streaming object
for event in streaming_object.graph.stream({"messages": messages}, thread):
    for v in event["event"]["values"]:
        print(v['messages'])

Los Angeles is warmer than San Francisco.
Actual message 1
Actual message 2
Actual message 3


## Streaming tokens

**Note** Ensure you have the required packages installed:
```py
# Install the Package Using pip (Recommended)
py -m pip install langgraph-checkpoint-sqlite
%pip install langgraph-checkpoint-sqlite

# Verify Installation  run:
py -m pip show langgraph-checkpoint-sqlite 

```

More info [`Installation`](https://pypi.org/project/langgraph-checkpoint-sqlite/)

In [14]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
memory = AsyncSqliteSaver.from_conn_string(":memory:")


In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import HumanMessage

# Define the custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.

Only look up information when you are sure of what you want. If you need to look up some information before asking a follow-up question, you are allowed to do that!
"""

# Initialize the Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="TavilySearch",
    func=tool.run,
    description="Search for information using Tavily."
)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the agent with the custom prompt
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  # Gracefully handle parsing errors
)

# Define the messages
messages = [HumanMessage(content="What is the weather in sf?")]

# Define the thread
thread = {"configurable": {"thread_id": "1"}}

# Run the agent with a query
try:
    response = agent.run(messages)
    print(response)
except ValueError as e:
    print(f"Agent failed to parse output: {e}")

# Since response is a string, you can print it directly
print(response)

The weather in San Francisco today is partly cloudy with a temperature of 55.0°F.
The weather in San Francisco today is partly cloudy with a temperature of 55.0°F.


 To enable token streaming, we  need to use agent.astream() (async streaming function) instead of agent.run(). Additionally, ChatOpenAI should be initialized with streaming=True. To sum up, to enalbel Token Streaming, thus in oder that the response streams token by token (like OpenAI API's streaming mode), let's consider the above code:

In [16]:
import nest_asyncio
import asyncio
import sys
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Apply nest_asyncio to fix nested event loops in Jupyter environments
nest_asyncio.apply()

# Initialize in-memory SQLite saver
memory = AsyncSqliteSaver.from_conn_string(":memory:")

# Define custom prompt
prompt = """You are a smart research assistant. Use the search engine to look up information.
You are allowed to make multiple calls (either together or in sequence).
When using a tool, always respond in this exact format:

Action: <tool_name>
Action Input: <tool_input>

For example:
Action: TavilySearch
Action Input: Find the latest AI news.
"""

# Initialize Tavily search tool
tool = TavilySearchResults(max_results=4)

# Wrap it as a LangChain Tool
search_tool = Tool(
    name="tavily_search_results_json",
    func=tool.run,
    description="Search for information using Tavily and return structured JSON results."
)

# Initialize the language model with streaming enabled
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, streaming=True)

# Create the agent
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    system_message=prompt,
    handle_parsing_errors=True  
)

# Define messages
messages = [HumanMessage(content="What is the current weather in San Francisco?")]

# Async function for streaming response
async def stream_response():
    async for event in agent.astream(messages[0].content):
        # 🔹 Debugging: Print full event to inspect structure
        print("\nDEBUG EVENT:", event)

        # Extract content properly
        if isinstance(event, dict) and "output" in event:
            content = event["output"].strip()
            if content:
                print(content.replace(" ", "|"), end="|", flush=True)  # Stream tokens in real-time
                sys.stdout.flush()  # Ensure immediate output

# Run the async function
await stream_response()


DEBUG EVENT: {'actions': [AgentAction(tool='tavily_search_results_json', tool_input="'current weather in San Francisco'", log="I should use the Tavily tool to search for the current weather in San Francisco.\nAction: tavily_search_results_json\nAction Input: 'current weather in San Francisco'")], 'messages': [AIMessage(content="I should use the Tavily tool to search for the current weather in San Francisco.\nAction: tavily_search_results_json\nAction Input: 'current weather in San Francisco'", additional_kwargs={}, response_metadata={})]}

DEBUG EVENT: {'steps': [AgentStep(action=AgentAction(tool='tavily_search_results_json', tool_input="'current weather in San Francisco'", log="I should use the Tavily tool to search for the current weather in San Francisco.\nAction: tavily_search_results_json\nAction Input: 'current weather in San Francisco'"), observation=[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'Un

**EXPLANATION OUTPUT** As it can be seen from the output now the Formats text are with | separators to simulate token streaming.

## Conclusion
This notebook provided a comprehensive guide to setting up an AI agent using LangChain, showcasing its capabilities in persistence, API interaction, and real-time data streaming. By following the steps outlined, you can create a robust system capable of managing and processing various tasks efficiently.